<a href="https://colab.research.google.com/github/cedro3/StyleGAN-Human/blob/main/stylegan_human.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **StyleGAN-Human demo**

In [ ]:
#@title **Setup**

# check gpu spec
! nvidia-smi -L

# get code from github
! git clone https://github.com/cedro3/StyleGAN-Human.git
%cd StyleGAN-Human

# install ninja
! wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
! sudo unzip ninja-linux.zip -d /usr/local/bin/
! sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force 

# install library
! pip install lpips
! pip install --upgrade gdown

# select tensorflow_version
%tensorflow_version 1.x 

# download pretrained models
! mkdir pretrained_models
import gdown
gdown.download('https://drive.google.com/uc?id=1FlAb1rYa0r_--Zj_ML8e6shmaF28hQb5', 'pretrained_models/stylegan2_1024.pkl', quiet=False)
gdown.download('https://drive.google.com/uc?id=125OG7SMkXI-Kf2aqiwLLHyCvSW-gZk3M', 'pretrained_models/ffhq.pkl', quiet=False)
gdown.download('https://drive.google.com/uc?id=1MduBgju5KFNrQfDLoQXJ_1_h5MnctCIG', 'pretrained_models/mmod_human_face_detector.dat', quiet=False)
gdown.download('https://drive.google.com/uc?id=1A82DnJBJzt8wI2J8ZrCK5fgHcQ2-tcWM', 'pretrained_models/shape_predictor_68_face_landmarks.dat', quiet=False)

# define function
from function import *


In [ ]:
#@title **Generate images**

seeds = "0-4" #@param {type:"string"}

! python generate.py --outdir=outputs/stylegan2_1024/\
                     --seeds=$seeds\
                     --trunc=0.7\
                     --network=pretrained_models/stylegan2_1024.pkl\
                     --version 2

# display_pic 
#display_pic('outputs/stylegan2_1024')

In [ ]:
#@title **interpolate images**

seeds = "97,9" #@param {type:"string"}

! python interpolation.py --network=pretrained_models/stylegan2_1024.pkl\
                          --seeds=$seeds\
                          --outdir=outputs/inter_gifs

# gif2mp4
! ffmpeg -y -i outputs/inter_gifs/latent_space_traversal.gif -pix_fmt yuv420p -loglevel error outputs/inter_gifs/inter.mp4

# display_mp4
display_mp4('outputs/inter_gifs/inter.mp4')

In [ ]:
#@title **Editing with InterfaceGAN, StyleSpace, and Sefa**

attr_name = 'upper_length' #@param ["upper_length", "botom_length"]
seeds = 61570 #@param {type:"integer"}

! python edit.py --outdir outputs/editing\
                 --network pretrained_models/stylegan2_1024.pkl\
                 --attr_name $attr_name\
                 --seeds $seeds

# display mp4
video_path = 'outputs/editing/video/'+attr_name+'_'+str(seeds)+'.mp4'
display_mp4(video_path) 

In [ ]:
#@title **style-mixing**

rows = "85,100,75,458,1500" #@param {type:"string"}
cols = "55,821,1789,293" #@param {type:"string"}
styles = "0-3" #@param {type:"string"}

! python style_mixing.py --outdir=outputs/stylemixing\
                         --rows=$rows\
                         --cols=$cols\
                         --network=pretrained_models/stylegan2_1024.pkl\
                         --styles=$styles

# display png
from IPython.display import Image,display_png
display_png(Image('outputs/stylemixing/grid.png'))

In [ ]:
#@title **Perform joint optimization and generate seamless images**

face_seed = "10" #@param {type:"string"}
body_seed = "89180" #@param {type:"string"}

!python insetgan.py --face_seed=$face_seed\
                    --body_seed=$body_seed\
                    --joint_steps=500\
                    --outdir outputs/insetgan\
                    --video 1

# display mp4
video_path = 'outputs/insetgan/'+face_seed.zfill(4)+'_'+body_seed.zfill(4)+'.mp4'
display_mp4(video_path)